In [ ]:
!pip install -U diffusers

In [ ]:
!pip install pyngrok
!pip install flask flask-ngrok

In [ ]:
!pip install flask_cors

In [ ]:
from diffusers import FluxPipeline as fp
import torch
from google.colab import userdata
from huggingface_hub import login

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
from psutil import virtual_memory

def get_mem():
  ram_gb = virtual_memory().total / 1e9
  print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

In [ ]:
HF_API_KEY = userdata.get('hf_botb')
login(token=HF_API_KEY)

In [ ]:
pipe = fp.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16)
pipe.enable_model_cpu_offload()

In [ ]:
company = "Home Depot"
product = "powertools"
benefit = "sturdiness"
media_co = "" # company that owns/produces the media being played (optional)
media = "show 'Dragon Ball Z'" # movie, series, etc. that is being played

In [ ]:
prompt = f"Create an advertisement for {company} {product} based on the {media_co} {media} and its characters."

In [ ]:
my_prompt = f"Create an advertisement for {company} {product} that references the {media_co} {media} and its characters.The ad must boldly display {company} and its logo."
# my_prompt = f"Create an advertisment for H-E-B supermarkets which is a Texas-based supermarket chain known for its strong customer loyalty and community involvement. It must reference sponge bob square pants shopping in a H-E-B supermarket. The ad must bodly display H-E-B and its logo."

In [ ]:
def gen_image(company, product, media_co, media):
  this_prompt = f"Create an advertisement for {company} {product} that references the {media_co} {media} and its characters.The ad must boldly display {company} and its logo."

  print(this_prompt)

  image = pipe(
      this_prompt,
      height=1024,
      width=1024,
      guidance_scale=5,
      num_inference_steps=50,
      max_sequence_length=512,
      generator=torch.Generator("cpu").manual_seed(10)
  ).images[0]

  image.save("flux-dev.png")

In [ ]:
# gen_image()

w

In [ ]:
from flask import Flask, request, jsonify, send_file
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
import torch
from diffusers import FluxPipeline as fp
import os
from google.colab import userdata
from flask_cors import CORS


NGROK_AUTH_TOKEN = userdata.get("ngrok_botb")

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

app = Flask(__name__)
CORS(app)  # Enable CORS globally

@app.route('/generate', methods=['OPTIONS', 'POST'])
def generate_ad():
    if request.method == 'OPTIONS':
        # Handle preflight request
        return jsonify({"message": "CORS preflight OK"}), 200

    data = request.json
    if not data:
        return jsonify({"error": "Invalid request body"}), 400

    print(data)
    company = data.get("company", "Default Company")
    product = data.get("product", "Default Product")
    benefit = data.get("benefit", "Default Benefit")
    media_co = data.get("media_co", "Default Media Company")
    media = data.get("media", "Default Media")

    print(f"[INFO] Received request to generate ad for {company} {product}.")
    print(f"[INFO] Using media inspiration: {media} from {media_co}.")

    try:
        gen_image(company, product, media_co, media)
        image_path = "flux-dev.png"
        print(f"[SUCCESS] Image successfully generated and saved as {image_path}.")
        return send_file(image_path, mimetype='image/png')

    except Exception as e:
        print(f"[ERROR] Failed to generate image: {str(e)}")
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    public_url = ngrok.connect(5000).public_url
    print(f"[INFO] Public API URL: {public_url}")
    app.run(threaded=False, processes=2)

[INFO] Public API URL: https://a2c2-34-45-4-90.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [16/Mar/2025 19:51:51] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Mar/2025 19:51:51] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
# DO NOT USE


"""
from flask import Flask, request, jsonify, send_file
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
import torch
from diffusers import FluxPipeline as fp
import os
from google.colab import userdata
from flask_cors import CORS


NGROK_AUTH_TOKEN = userdata.get("ngrok_botb")

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

CORS(app)
app = Flask(__name__)

print("[INFO] Loading AI model...")
pipe = fp.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16)
pipe.enable_model_cpu_offload()
print("[INFO] AI model loaded successfully!")

@app.route('/generate', methods=['POST'])
def generate_ad():
    data = request.json

    company = data.get("company", "Default Company")
    product = data.get("product", "Default Product")
    benefit = data.get("benefit", "Default Benefit")
    media_co = data.get("media_co", "Default Media Company")
    media = data.get("media", "Default Media")

    # prompt = f"Create an advertisement for {company} {product} based on the {media_co} {media}."

    print(f"[INFO] Received request to generate ad for {company} {product}.")
    print(f"[INFO] Using media inspiration: {media} from {media_co}.")

    try:
        gen_img()

        image_path = "flux-dev.png"

        print(f"[SUCCESS] Image successfully generated and saved as {image_path}.")
        return send_file(image_path, mimetype='image/png')

    except Exception as e:
        print(f"[ERROR] Failed to generate image: {str(e)}")
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':

    print(f"[INFO] Public API URL: {public_url}")
    app.run(host='0.0.0.0', port=8080)

"""

In [ ]:
# DO NOT USE

"""
from flask import Flask, request, jsonify, send_file
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
import torch
from diffusers import FluxPipeline as fp
import os
from google.colab import userdata


NGROK_AUTH_TOKEN = userdata.get("ngrok_botb")

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

app = Flask(__name__)
run_with_ngrok(app)

print("[INFO] Loading AI model...")
pipe = fp.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16)
pipe.enable_model_cpu_offload()
print("[INFO] AI model loaded successfully!")

# @app.route('/generate', methods=['OPTIONS'])
# def test():
#     print("Received OPTIONS request")

@app.route('/generate', methods=['OPTIONS'])
def generate_ad():
    data = request.json
    print(data)

    company = data.get("company", "Default Company")
    product = data.get("product", "Default Product")
    benefit = data.get("benefit", "Default Benefit")
    media_co = data.get("media_co", "Default Media Company")
    media = data.get("media", "Default Media")

    # prompt = f"Create an advertisement for {company} {product} based on the {media_co} {media}."

    print(f"[INFO] Received request to generate ad for {company} {product}.")
    print(f"[INFO] Using media inspiration: {media} from {media_co}.")

    try:
        gen_img()

        image_path = "flux-dev.png"

        print(f"[SUCCESS] Image successfully generated and saved as {image_path}.")
        return send_file(image_path, mimetype='image/png')

    except Exception as e:
        print(f"[ERROR] Failed to generate image: {str(e)}")
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    public_url = ngrok.connect(5000).public_url
    print(f"[INFO] Public API URL: {public_url}")
    app.run()
"""

In [ ]:
!lsof -i :8080

COMMAND PID USER   FD   TYPE  DEVICE SIZE/OFF NODE NAME
node      7 root   21u  IPv6   20647      0t0  TCP *:8080 (LISTEN)
node      7 root   26u  IPv6 1419923      0t0  TCP cea117fa1967:8080->172.28.0.1:58760 (ESTABLISHED)
node      7 root   28u  IPv6 1464800      0t0  TCP cea117fa1967:8080->172.28.0.1:35190 (ESTABLISHED)
node      7 root   30u  IPv6 1472051      0t0  TCP cea117fa1967:8080->172.28.0.1:46758 (ESTABLISHED)


In [ ]:
!kill 7